In [1]:
%pip install jaal


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd


from aperturedb.CommonLibrary import (
    execute_query,
    create_connector
)
from aperturedb.Utils import Utils


In [3]:
client=create_connector()
utils = Utils(client)
utils.summary()


================== Summary ==================
Database: garfield
Version: 0.18.1
Status:  0
Info:    OK
------------------ Entities -----------------
Total entities types:    2
Movie               
  Total elements: 4803
    Number   | budget      |      4803 (100%)
I   String   | id          |      4803 (100%)
  ! Number   | movie_id    |      4803 (100%)
    String   | overview    |      4803 (100%)
    Number   | popularity  |      4803 (100%)
    String   | title       |      4803 (100%)
Professional        
  Total elements: 104842
    Number   | gender  |    104842 (100%)
I   Number   | id      |    104842 (100%)
    String   | name    |    104842 (100%)
---------------- Connections ----------------
Total connections types: 2
CAST                
  Movie ====> Professional
  Total elements: 106257
I   Number   | cast_id    |    106257 (100%)
    String   | character  |    106257 (100%)
CREW                
  Movie ====> Professional
  Total elements: 129581
  ! String   | credit_

In [4]:
movie_ids = [
                    64685,
                    9800,
                    10193,
                    5255,
                    140823,
                    5516,
                    13448,
                    20763,
                    591,
                    862,
                    83542,
                    863,
                    8358,
                    9591,
                    59861,
                    4147,
                    594,
                    6538,
                    35,
                    920,
                    857,
                    568,
                    9489,
                    497,
                    13508,
                    2619,
                    13,
                    109424,
                    640,
                    2280,
                    11287,
                    296098]

In [5]:
q = [
    {
        "FindEntity": {
            "with_class": "MOVIE",
            "constraints": {
                "movie_id": ["in",  movie_ids]
            },
            "_ref": 1,
            "results": {
                "all_properties": True,
            },
        }
    },
    {
        "FindEntity": {
            "with_class": "PROFESSIONAL",
            "is_connected_to": {
                "ref": 1,
            },
            "results": {
                "all_properties": True,
                "group_by_source": True,
            },
        }
    },
]

result, response, b = execute_query(client, q)

nodes = []
nodeset = set()
edges = set()
for e in response[0]['FindEntity']['entities']:
    nodes.append({
        "id": e['_uniqueid'],
        "label": e['title'],
        "budget": e['budget'],
        "popularity": e['popularity'],
        "type": "MOVIE"
    })
    nodeset.add(e['_uniqueid'])
for src in response[0]['FindEntity']['entities']:
    src_id = src['_uniqueid']
    for e in response[1]['FindEntity']['entities'][src_id]:
        if e['_uniqueid'] not in nodeset:
            nodes.append({
                "id": e['_uniqueid'],
                "label": e['name'],
                "gender": e['gender'],
                "budget": 10000,
                "type": "PROFESSIONAL"
            })
            nodeset.add(e['_uniqueid'])
        edges.add((src_id, e['_uniqueid']))

nodes_df = pd.DataFrame(nodes)
nodes_df

,id,label,budget,popularity,type,gender
0,11.179.53800,Philadelphia,26000000,44.301745,MOVIE,NaN
1,11.529.53808,Extremely Loud & Incredibly Close,40000000,31.066874,MOVIE,NaN
2,11.627.53810,Saving Private Ryan,70000000,76.041867,MOVIE,NaN
3,11.634.53810,Apollo 13,52000000,68.140214,MOVIE,NaN
4,11.637.53810,You've Got Mail,65000000,28.540267,MOVIE,NaN
...,...,...,...,...,...,...
2453,12.104030.53960,Eben Ostby,10000,NaN,PROFESSIONAL,0.0
2454,12.104031.53960,Jessica Giampietro McMackin,10000,NaN,PROFESSIONAL,0.0
2455,12.104032.53960,Kiel Murray,10000,NaN,PROFESSIONAL,0.0
2456,12.104033.53960,Phil Lorin,10000,NaN,PROFESSIONAL,0.0


In [6]:
cq = []
i = 0
for src, dst in edges:
    cq += [
        {
            "FindEntity": {
                "_ref": 1 + i,
                "constraints": {
                    "_uniqueid": ["==", src],
                },
                "results": {
                    "all_properties": True,
                },
            }
        },
        {
            "FindEntity": {
                "_ref": 2 + i,
                "constraints": {
                    "_uniqueid": ["==", dst],
                },
                "results": {
                    "all_properties": True,
                },
            }
        },
        {
            "FindConnection": {
                "src": 1 + i,
                "dst": 2 + i,
                "with_class": "CAST",
                "results": {
                    "all_properties": True,
                },
            }
        },
    ]
    i += 2

connections = set()

result, response, b = execute_query(client, cq)
for i in range(0, len(response), 3):
    src = response[i]["FindEntity"]["entities"][0]["_uniqueid"]
    dst = response[i + 1]["FindEntity"]["entities"][0]["_uniqueid"]
    if "connections" not in response[i + 2]["FindConnection"]:
        continue
    for c in response[i + 2]["FindConnection"]["connections"]:
        if "character" in c:
            connections.add((src, dst, c["character"]))
            break
        else:
            id = f'{c["job"]}_{c["department"]}'
            connections.add((src, dst, id))
            break



edge_df = pd.json_normalize(
    [
        {"from": s, "to": d, "label": id, "id": f"{s}_{d}_{id}"}
        for s, d, id in connections
    ]
)
edge_df

,from,to,label,id
0,11.2582.53862,12.68525.53862,Co-Pilot,11.2582.53862_12.68525.53862_Co-Pilot
1,11.4687.53920,12.102621.53920,Classroom Boy #1,11.4687.53920_12.102621.53920_Classroom Boy #1
2,11.4687.53920,12.8775.53802,Mary Donovan,11.4687.53920_12.8775.53802_Mary Donovan
3,11.4097.53894,12.2663.53800,Young Rufus Sixsmith / Old Rufus Sixsmith / Nu...,11.4097.53894_12.2663.53800_Young Rufus Sixsmi...
4,11.627.53810,12.21072.53810,Jean's Daughter,11.627.53810_12.21072.53810_Jean's Daughter
...,...,...,...,...
955,11.742.53812,12.15969.53806,Lotso (voice),11.742.53812_12.15969.53806_Lotso (voice)
956,11.4097.53894,12.17792.53808,Adam Ewing / Poor Hotel Guest / Megan's Dad / ...,11.4097.53894_12.17792.53808_Adam Ewing / Poor...
957,11.2582.53862,12.68491.53862,Young Patient,11.2582.53862_12.68491.53862_Young Patient
958,11.4760.53960,12.5826.53800,Himself (voice),11.4760.53960_12.5826.53800_Himself (voice)


In [7]:
from jaal import Jaal
Jaal(edge_df, nodes_df).plot()

Parsing the data...Done


/Users/gsaluja/.virtualenvs/croissant/lib/python3.11/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


No trigger
